In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import pandas as pd
from pandas import DataFrame
from pandas import Series
from netCDF4 import Dataset
from netCDF4 import num2date
from netCDF4 import date2num
import datetime
import os
import sys
import os.path
import glob

In [2]:
# Reading in model output files, from three different applications, fabm0d,
# gotm1d and gotmlake
output_dir= os.path.normpath(os.getcwd() + os.sep + os.pardir+"/output")
fabm0d=['pclake-fabm0d-2m.nc','pclake-fabm0d-5m.nc','pclake-fabm0d-10m.nc','pclake-fabm0d-20m.nc']
gotm1d=['pclake-gotm1d-2m.nc','pclake-gotm1d-5m.nc','pclake-gotm1d-10m.nc','pclake-gotm1d-20m.nc']
gotmlake=['pclake-gotmlake-2m.nc','pclake-gotmlake-5m.nc','pclake-gotmlake-10m.nc','pclake-gotmlake-20m.nc']

In [3]:
# Set ploting time intervals(keep matplotlib datetime format)
# Since 3 applications has the same time intervals, so use anyone of them is fine
# Set ploting time intervals(keep matplotlib datetime format)
path=os.path.join(output_dir, fabm0d[0])
# convert NetCDF data to Dataset, to read time
time_nc=Dataset(path, mode='r')
time        = time_nc.variables['time']
units       = time_nc.variables['time'].units
valid_times = num2date(time[:], units=units).tolist()
# Set the start and stop point for ploting interval
start=valid_times.index(datetime.datetime(2015, 6, 1))
stop=valid_times.index(datetime.datetime(2016, 9, 1))
# Time for extracting data
time=valid_times[start:stop]
# ploting x axis
dt=datetime.date
# Setting period, and model output intervals is 1day
time_t=mdates.drange(dt(2015,1,1), dt(2016,1,1), datetime.timedelta(days=1))

In [4]:
# Read in gotm1d data, get the vertical avarage value, and store in nc Dataset form
# creat dict for store output of wanted fabm0d data
Tm_0d=[];O2_0d=[];sPO4W_0d=[];sNO3W_0d=[];
Phy_0d=[];Zoo_0d=[];Veg_0d=[];Fish_0d=[];
sPO4S_0d=[];sNO3S_0d=[];Ben_0d = [];

for f_0d in fabm0d:
    path=os.path.join(output_dir, f_0d)
    fabm0d_nc=Dataset(path, mode='r')
# create empty lists for storging extracted and treated variables
    Tm   = fabm0d_nc.variables['temp'][start:stop,0,0]
#    PAR  = fabm0d_nc.variables['phytoplankton_water_partop'][start:stop,0,0]
#    PAR  = fabm0d_nc.variables['phytoplankton_water_phypar'][start:stop,0,0]
    O2   = fabm0d_nc.variables['abiotic_water_sO2W'][start:stop,0,0]
#    TN   = fabm0d_nc.variables['pclake_totN_calculator_result'][start:stop,0,0]
#    TP   = fabm0d_nc.variables['pclake_totP_calculator_result'][start:stop,0,0]
    PO4  = fabm0d_nc.variables['abiotic_water_sPO4W'][start:stop,0,0];
    NO3  = fabm0d_nc.variables['abiotic_water_sNO3W'][start:stop,0,0];
    DBlue=fabm0d_nc.variables['phytoplankton_water_sDBlueW'][start:stop,0,0]
    DGren=fabm0d_nc.variables['phytoplankton_water_sDGrenW'][start:stop,0,0]
    DDiat=fabm0d_nc.variables['phytoplankton_water_sDDiatW'][start:stop,0,0]
    DPhyt=np.add(np.add(DBlue,DGren),DDiat)
#    Phy  = fabm0d_nc.variables['pclake_chla_calculator_result'][start:stop,0,0]
    Zoo  = fabm0d_nc.variables['zooplankton_sDZoo'][start:stop,0,0];
    Fish = fabm0d_nc.variables['fish_sDFiAd'][start:stop,0,0]+ \
    fabm0d_nc.variables['fish_sDFiJv'][start:stop,0,0];
#   sediment nutrients
    Veg=fabm0d_nc.variables['macrophytes_sDVeg'][start:stop,0,0]
    Ben=fabm0d_nc.variables['zoobenthos_sDBent'][start:stop,0,0]
    PO4S=fabm0d_nc.variables['abiotic_sediment_sPO4S'][start:stop,0,0];
    NO3S=fabm0d_nc.variables['abiotic_sediment_sNO3S'][start:stop,0,0];   
# Get anually average values
    Tm_0d.append(np.mean(Tm[:]));
#    PAR_0d.append(np.mean(PAR[:]))
    O2_0d.append(np.mean(O2[:]));
    sPO4W_0d.append(np.mean(PO4[:]));
    sNO3W_0d.append(np.mean(NO3[:]));
    Phy_0d.append(np.mean(DPhyt[:]));
    Zoo_0d.append(np.mean(Zoo[:]));
    Veg_0d.append(np.mean(Veg[:]));
    Fish_0d.append(np.mean(Fish[:]));
#    TN_0d.append(np.mean(TN[:]))
#    TP_0d.append(np.mean(TP[:]))
#    Phy_0d.append(np.mean(Phy[:]))
    sPO4S_0d.append(np.mean(PO4S[:]));
    sNO3S_0d.append(np.mean(NO3S[:]));
    Ben_0d.append(np.mean(Ben[:]));
      
Ave_0d = np.row_stack((Tm_0d,O2_0d,sPO4W_0d,sNO3W_0d,Phy_0d,Zoo_0d,Veg_0d,Fish_0d,sPO4S_0d,sNO3S_0d,Ben_0d));
df_0d  = DataFrame(Ave_0d,columns=['2m','5m','10m','20m'],index=['Tm','O2','PO4W','NO3W','Phy','Zoo','Veg','Fish','PO4S','NO3S','Ben'])

In [5]:
# Get gotm1d data
#group1
Tm_1d_surf=[];Tm_1d_bott=[];Tm_1d_mean=[];
O2_1d_surf=[];O2_1d_bott=[];O2_1d_mean=[];
PO4_1d_surf=[];PO4_1d_bott=[];PO4_1d_mean=[];
NO3_1d_surf=[];NO3_1d_bott=[];NO3_1d_mean=[];
#group2
Phy_1d_surf=[];Phy_1d_bott=[];Phy_1d_mean=[];
Zoo_1d_surf=[];Zoo_1d_bott=[];Zoo_1d_mean=[];
Veg_mean=[];
Fish_mean=[];
#group3
PO4S_mean=[];
NO3S_mean=[];
Ben_mean=[];

for f_1d in gotm1d:
    path=os.path.join(output_dir, f_1d)
    gotm1d_nc=Dataset(path, mode='r')
#   Get surface value   
    Tm_surf   = gotm1d_nc.variables['temp'][start:stop,-1,0,0];
    O2_surf   = gotm1d_nc.variables['abiotic_water_sO2W'][start:stop,-1,0,0];
    PO4_surf   = gotm1d_nc.variables['abiotic_water_sPO4W'][start:stop,-1,0,0];
    NO3_surf   = gotm1d_nc.variables['abiotic_water_sNO3W'][start:stop,-1,0,0];
    DBlue_surf = gotm1d_nc.variables['phytoplankton_water_sDBlueW'][start:stop,-1,0,0];
    DGren_surf = gotm1d_nc.variables['phytoplankton_water_sDGrenW'][start:stop,-1,0,0];
    DDiat_surf = gotm1d_nc.variables['phytoplankton_water_sDDiatW'][start:stop,-1,0,0];
    Phy_surf  = np.add(np.add(DBlue_surf,DGren_surf),DDiat_surf);
    Zoo_surf  = gotm1d_nc.variables['zooplankton_sDZoo'][start:stop,-1,0,0];
#    Fish_surf = gotm1d_nc.variables['fish_sDFiAd'][start:stop,-1,0,0]*0.4 + \
#                gotm1d_nc.variables['fish_sDFiJv'][start:stop,-1,0,0]*0.4  
#    Fish_surf = gotm1d_nc.variables['fish_sDFiAd'][start:stop,0,0] + \
#                gotm1d_nc.variables['fish_sDFiJv'][start:stop,0,0]
#   Get bottom value
    Tm_bott   = gotm1d_nc.variables['temp'][start:stop,0,0,0];
    O2_bott   = gotm1d_nc.variables['abiotic_water_sO2W'][start:stop,0,0,0];
    PO4_bott   = gotm1d_nc.variables['abiotic_water_sPO4W'][start:stop,0,0,0];
    NO3_bott   = gotm1d_nc.variables['abiotic_water_sNO3W'][start:stop,0,0,0];
    DBlue_bott = gotm1d_nc.variables['phytoplankton_water_sDBlueW'][start:stop,0,0,0];
    DGren_bott = gotm1d_nc.variables['phytoplankton_water_sDGrenW'][start:stop,0,0,0];
    DDiat_bott = gotm1d_nc.variables['phytoplankton_water_sDDiatW'][start:stop,0,0,0];
    Phy_bott  = np.add(np.add(DBlue_bott,DGren_bott),DDiat_bott);
    Zoo_bott  = gotm1d_nc.variables['zooplankton_sDZoo'][start:stop,0,0,0]
#    Fish_bott = gotm1d_nc.variables['foodweb_water_sDFiAd'][start:stop,0,0,0]*0.4 + \
#                gotm1d_nc.variables['foodweb_water_sDFiJv'][start:stop,0,0,0]*0.4    
#    Fish_bott = gotm1d_nc.variables['fish_sDFiAd'][start:stop,0,0] + \
#                gotm1d_nc.variables['fish_sDFiJv'][start:stop,0,0]    

#  Get vertical average    
    Tm_mean=[];O2_mean=[];PO4_mean=[];NO3_mean=[];Phy_mean=[];Zoo_mean=[];
    i=start
    for t in time:
        Tm_mean.append(np.mean(gotm1d_nc.variables['temp'][i,:,0,0]));
        O2_mean.append(np.mean(gotm1d_nc.variables['abiotic_water_sO2W'][i,:,0,0]));
        PO4_mean.append(np.mean(gotm1d_nc.variables['abiotic_water_sPO4W'][i,:,0,0]))
        NO3_mean.append(np.mean(gotm1d_nc.variables['abiotic_water_sNO3W'][i,:,0,0]))
        Phy_mean.append(np.mean(gotm1d_nc.variables['phytoplankton_water_sDBlueW'][i,:,0,0])
                        +np.mean(gotm1d_nc.variables['phytoplankton_water_sDDiatW'][i,:,0,0])
                        +np.mean(gotm1d_nc.variables['phytoplankton_water_sDGrenW'][i,:,0,0]))
        Zoo_mean.append(np.mean(gotm1d_nc.variables['zooplankton_sDZoo'][i,:,0,0]))
        i=i+1
#   sediment nutrients
    Fish = fabm0d_nc.variables['fish_sDFiAd'][start:stop,0,0]+ \
    fabm0d_nc.variables['fish_sDFiJv'][start:stop,0,0];
    Veg=gotm1d_nc.variables['macrophytes_sDVeg'][start:stop,0,0]
    Ben=gotm1d_nc.variables['zoobenthos_sDBent'][start:stop,0,0]
    PO4S=gotm1d_nc.variables['abiotic_sediment_sPO4S'][start:stop,0,0];
    NO3S=gotm1d_nc.variables['abiotic_sediment_sNO3S'][start:stop,0,0];  
    
# Assemble surface variables
    Tm_1d_surf.append(np.mean(Tm_surf[:]));
    O2_1d_surf.append(np.mean(O2_surf[:]));
    PO4_1d_surf.append(np.mean(PO4_surf[:]));
    NO3_1d_surf.append(np.mean(NO3_surf[:]));
    Phy_1d_surf.append(np.mean(Phy_surf[:]));
    Zoo_1d_surf.append(np.mean(Zoo_surf[:]));    
# Assemble bottom variables    
    Tm_1d_bott.append(np.mean(Tm_bott[:])); 
    O2_1d_bott.append(np.mean(O2_bott[:]));
    PO4_1d_bott.append(np.mean(PO4_bott[:]));
    NO3_1d_bott.append(np.mean(NO3_bott[:]));
    Phy_1d_bott.append(np.mean(Phy_bott[:]));
    Zoo_1d_bott.append(np.mean(Zoo_bott[:]));
# Assemble mean values
    Tm_1d_mean.append(np.mean(Tm_mean[:]));
    O2_1d_mean.append(np.mean(O2_mean[:]));
    PO4_1d_mean.append(np.mean(PO4_mean[:]));
    NO3_1d_mean.append(np.mean(NO3_mean[:]));
    Phy_1d_mean.append(np.mean(Phy_mean[:]));
    Zoo_1d_mean.append(np.mean(Zoo_mean[:]));
# Sediment variables
    Fish_mean.append(np.mean(Fish[:]));
    Veg_mean.append(np.mean(Veg[:]));
    Ben_mean.append(np.mean(Ben[:]));
    PO4S_mean.append(np.mean(PO4S[:]));
    NO3S_mean.append(np.mean(NO3S[:]));

# Put three different layer together   
Tm_1d= np.row_stack((Tm_1d_mean,Tm_1d_surf,Tm_1d_bott));
O2_1d= np.row_stack((O2_1d_mean,O2_1d_surf,O2_1d_bott));
PO4_1d= np.row_stack((PO4_1d_mean,PO4_1d_surf,PO4_1d_bott));
NO3_1d= np.row_stack((NO3_1d_mean,NO3_1d_surf,NO3_1d_bott));
Phy_1d= np.row_stack((Phy_1d_mean,Phy_1d_surf,Phy_1d_bott));
Zoo_1d= np.row_stack((Zoo_1d_mean,Zoo_1d_surf,Zoo_1d_bott));
Fish_1d=np.row_stack((Fish_mean,Fish_mean,Fish_mean));
Veg_1d= np.row_stack((Veg_mean,Veg_mean,Veg_mean));
Ben_1d=np.row_stack((Ben_mean,Ben_mean,Ben_mean)); 
PO4S_1d=np.row_stack((PO4S_mean,PO4S_mean,PO4S_mean)); 
NO3S_1d=np.row_stack((NO3S_mean,NO3S_mean,NO3S_mean)); 

# assemble all data together
Ave_1d = np.row_stack((Tm_1d,O2_1d,PO4_1d,NO3_1d,Phy_1d,Zoo_1d,Fish_1d,Veg_1d,PO4S_1d,NO3S_1d,Ben_1d))
df_1d  = DataFrame(Ave_1d,columns=['2m','5m','10m','20m'],index=['Tm','Tm_surf','Tm_bott',
                                                                 'O2','O2_surf','O2_bott',
                                                                 'PO4W','PO4W_surf','PO4W_bott',
                                                                 'NO3W','NO3W_surf','NO3W_bott',
                                                                 'Phy','Phy_surf','Phy_bott',
                                                                 'Zoo','Zoo_surf','Zoo_bott',
                                                                 'Fish','Fish_surf','Fish_bott',
                                                                'Veg','Veg_surf','Veg_bott',
                                                                'PO4S','PO4S_surf','PO4S_bott',
                                                                'NO3S','NO3S_surf','NO3S_bott',
                                                                'Ben','Ben_surf','Ben_bott'])

In [6]:
# get gotmlake data
Tm_lake_surf=[];Tm_lake_bott=[];Tm_lake_mean=[];
O2_lake_surf=[];O2_lake_bott=[];O2_lake_mean=[];
PO4_lake_surf=[];PO4_lake_bott=[];PO4_lake_mean=[];
NO3_lake_surf=[];NO3_lake_bott=[];NO3_lake_mean=[];
Phy_lake_surf=[];Phy_lake_bott=[];Phy_lake_mean=[];
Zoo_lake_surf=[];Zoo_lake_bott=[];Zoo_lake_mean=[];
#sediment variables
Fish_lake_surf=[];Fish_lake_bott=[];Fish_lake_mean=[];
#group3
Veg_lake_surf=[];Veg_lake_bott=[];Veg_lake_mean=[];
Ben_lake_surf=[];Ben_lake_bott=[];Ben_lake_mean=[];
PO4S_lake_surf=[];PO4S_lake_bott=[];PO4S_lake_mean=[];
NO3S_lake_surf=[];NO3S_lake_bott=[];NO3S_lake_mean=[];
# Get the volumn fraction for each depth in gotmlake
# calculated according to hypsograph data in excel.
f_Vn_2m=[0.04,0.12,0.20,0.28,0.36]
f_Vn_5m=[0.01,0.02,0.03,0.04,0.05,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.15]
f_Vn_10m=[0.0025,0.0056,0.0087,0.0119,0.0150,0.0181,0.0212,0.0244,0.0275,
          0.0306,0.0337,0.0369,0.0400,0.0431,0.0463,0.0494,0.0525,0.0556,
          0.0588,0.0619,0.0650,0.0681,0.0713,0.0744,0.0775]
f_Vn_20m=[0.000854443,0.001635895,0.002417346,0.003198797,0.003980249,0.0047617,
          0.005543151,0.006324602,0.007106054,0.007887505,0.008668956,0.009450408,
          0.010231859,0.01101331,0.011794761,0.012576213,0.013357664,0.014139115,
          0.014920567,0.015702018,0.016483469,0.01726492,0.018046372,0.018827823,
          0.019609274,0.020390726,0.021172177,0.021953628,0.02273508,0.023516531,
          0.024297982,0.025079433,0.025860885,0.026642336,0.027423787,0.028205239,
          0.02898669,0.029768141,0.030549592,0.031331044,0.032112495,0.032893946,
          0.033675398,0.034456849,0.0352383,0.036019751,0.036801203,0.037582654,
          0.038364105,0.039145557]
for f_lake in gotmlake:
    path=os.path.join(output_dir, f_lake)
    gotmlake_nc=Dataset(path, mode='r')
    # Get the f_lvl for different depth
    lvl=len(gotmlake_nc.variables['temp'][0])
    if lvl==5:
        f_lvl=f_Vn_2m
    elif lvl== 13:
        f_lvl=f_Vn_5m
    elif lvl==25:
        f_lvl=f_Vn_10m
    elif lvl==50:
        f_lvl=f_Vn_20m
#   Get surface value   
    Tm_surf   = gotmlake_nc.variables['temp'][start:stop,-1,0,0]
    O2_surf   = gotmlake_nc.variables['abiotic_water_sO2W'][start:stop,-1,0,0]
    PO4_surf   = gotmlake_nc.variables['abiotic_water_sPO4W'][start:stop,-1,0,0]
    NO3_surf   = gotmlake_nc.variables['abiotic_water_sNO3W'][start:stop,-1,0,0]
    DBlue_surf = gotmlake_nc.variables['phytoplankton_water_sDBlueW'][start:stop,-1,0,0];
    DGren_surf = gotmlake_nc.variables['phytoplankton_water_sDGrenW'][start:stop,-1,0,0];
    DDiat_surf = gotmlake_nc.variables['phytoplankton_water_sDDiatW'][start:stop,-1,0,0];
    Phy_surf  = np.add(np.add(DBlue_surf,DGren_surf),DDiat_surf);
    Zoo_surf  = gotmlake_nc.variables['zooplankton_sDZoo'][start:stop,-1,0,0]
    Fish_surf = gotmlake_nc.variables['fish_sDFiAd'][start:stop,-1,0,0] + \
                gotmlake_nc.variables['fish_sDFiJv'][start:stop,-1,0,0];
    Veg_surf = gotmlake_nc.variables['macrophytes_sDVeg'][start:stop,-1,0,0];
    PO4S_surf =gotmlake_nc.variables['abiotic_sediment_sPO4S'][start:stop,-1,0,0]; 
    NO3S_surf =gotmlake_nc.variables['abiotic_sediment_sNO3S'][start:stop,-1,0,0]; 
    Ben_surf =gotmlake_nc.variables['zoobenthos_sDBent'][start:stop,-1,0,0]; 
    
#   Get bottom value
    Tm_bott   = gotmlake_nc.variables['temp'][start:stop,0,0,0]
    O2_bott   = gotmlake_nc.variables['abiotic_water_sO2W'][start:stop,0,0,0]
    PO4_bott   = gotmlake_nc.variables['abiotic_water_sPO4W'][start:stop,0,0,0]
    NO3_bott   = gotmlake_nc.variables['abiotic_water_sNO3W'][start:stop,0,0,0]
    DBlue_bott = gotmlake_nc.variables['phytoplankton_water_sDBlueW'][start:stop,0,0,0];
    DGren_bott = gotmlake_nc.variables['phytoplankton_water_sDGrenW'][start:stop,0,0,0];
    DDiat_bott = gotmlake_nc.variables['phytoplankton_water_sDDiatW'][start:stop,0,0,0];
    Phy_bott  = np.add(np.add(DBlue_bott,DGren_bott),DDiat_bott);
    Zoo_bott  = gotmlake_nc.variables['zooplankton_sDZoo'][start:stop,0,0,0]
    Fish_bott = gotmlake_nc.variables['fish_sDFiAd'][start:stop,0,0,0] + \
                gotmlake_nc.variables['fish_sDFiJv'][start:stop,0,0,0];
    Veg_bott = gotmlake_nc.variables['macrophytes_sDVeg'][start:stop,0,0,0];
    PO4S_bott =gotmlake_nc.variables['abiotic_sediment_sPO4S'][start:stop,0,0,0]; 
    NO3S_bott =gotmlake_nc.variables['abiotic_sediment_sNO3S'][start:stop,0,0,0]; 
    Ben_bott =gotmlake_nc.variables['zoobenthos_sDBent'][start:stop,0,0,0]; 
#  Get vertical wieghted average       
    Tm_mean=[];PAR_mean=[];O2_mean=[];PO4_mean=[];NO3_mean=[];Phy_mean=[];Zoo_mean=[];Fish_mean=[];
    DBlue_mean=[];DDiat_mean=[];DGren_mean=[];Veg_mean=[];PO4S_mean=[];NO3S_mean=[];Ben_mean=[];
    i=start
    for t in time:
        Tm_mean.append(np.sum(gotmlake_nc.variables['temp'][i,:,0,0]*f_lvl[:]));
        O2_mean.append(np.sum(gotmlake_nc.variables['abiotic_water_sO2W'][i,:,0,0]*f_lvl[:]));
        PO4_mean.append(np.sum(gotmlake_nc.variables['abiotic_water_sPO4W'][i,:,0,0]*f_lvl[:]));
        NO3_mean.append(np.sum(gotmlake_nc.variables['abiotic_water_sNO3W'][i,:,0,0]*f_lvl[:]));
        DBlue_mean.append(np.sum(gotmlake_nc.variables['phytoplankton_water_sDBlueW'][i,:,0,0]*f_lvl[:]));
        DDiat_mean.append(np.sum(gotmlake_nc.variables['phytoplankton_water_sDDiatW'][i,:,0,0]*f_lvl[:]));
        DGren_mean.append(np.sum(gotmlake_nc.variables['phytoplankton_water_sDGrenW'][i,:,0,0]*f_lvl[:]));
        Zoo_mean.append(np.sum(gotmlake_nc.variables['zooplankton_sDZoo'][i,:,0,0]*f_lvl[:]));
        Fish_mean.append(np.mean(gotmlake_nc.variables['fish_sDFiAd'][i,:,0,0]) 
                                 + np.mean(gotmlake_nc.variables['fish_sDFiJv'][i,:,0,0]));
        Veg_mean.append(np.mean(gotmlake_nc.variables['macrophytes_sDVeg'][i,:,0,0]));
        PO4S_mean.append(np.mean(gotmlake_nc.variables['abiotic_sediment_sPO4S'][i,:,0,0]));
        NO3S_mean.append(np.mean(gotmlake_nc.variables['abiotic_sediment_sNO3S'][i,:,0,0]));
        Ben_mean.append(np.mean(gotmlake_nc.variables['zoobenthos_sDBent'][i,:,0,0]));        
        i=i+1
    Phy_mean  = np.add(np.add(DBlue_mean,DGren_mean),DDiat_mean);   
# Assemble surface variables
    Tm_lake_surf.append(np.mean(Tm_surf[:]));
    O2_lake_surf.append(np.mean(O2_surf[:]));
    PO4_lake_surf.append(np.mean(PO4_surf[:]));
    NO3_lake_surf.append(np.mean(NO3_surf[:]));
    Phy_lake_surf.append(np.mean(Phy_surf[:]));
    Zoo_lake_surf.append(np.mean(Zoo_surf[:]));
    Fish_lake_surf.append(np.mean(Fish_surf[:]));
    Veg_lake_surf.append(np.mean(Veg_surf[:]));
    PO4S_lake_surf.append(np.mean(PO4S_surf[:]));
    NO3S_lake_surf.append(np.mean(NO3S_surf[:]));
    Ben_lake_surf.append(np.mean(Ben_surf[:]));    
# Assemble bottom variables    
    Tm_lake_bott.append(np.mean(Tm_bott[:])); 
    O2_lake_bott.append(np.mean(O2_bott[:]));
    PO4_lake_bott.append(np.mean(PO4_bott[:]));
    NO3_lake_bott.append(np.mean(NO3_bott[:]));
    Phy_lake_bott.append(np.mean(Phy_bott[:]));
    Zoo_lake_bott.append(np.mean(Zoo_bott[:]));
    Fish_lake_bott.append(np.mean(Fish_bott[:]));
    Veg_lake_bott.append(np.mean(Veg_bott[:]));
    PO4S_lake_bott.append(np.mean(PO4S_bott[:]));
    NO3S_lake_bott.append(np.mean(NO3S_bott[:]));
    Ben_lake_bott.append(np.mean(Ben_bott[:]));
# Assemble mean values
    Tm_lake_mean.append(np.mean(Tm_mean[:]));
    O2_lake_mean.append(np.mean(O2_mean[:]));
    PO4_lake_mean.append(np.mean(PO4_mean[:]));
    NO3_lake_mean.append(np.mean(NO3_mean[:]));
    Phy_lake_mean.append(np.mean(Phy_mean[:]));
    Zoo_lake_mean.append(np.mean(Zoo_mean[:]));
    Fish_lake_mean.append(np.mean(Fish_mean[:])); 
    Veg_lake_mean.append(np.mean(Veg_mean[:]));
    PO4S_lake_mean.append(np.mean(PO4S_mean[:]));
    NO3S_lake_mean.append(np.mean(NO3S_mean[:]));
    Ben_lake_mean.append(np.mean(Ben_mean[:]));        
# Put three different layer together   
Tm_lake= np.row_stack((Tm_lake_mean,Tm_lake_surf,Tm_lake_bott));
O2_lake= np.row_stack((O2_lake_mean,O2_lake_surf,O2_lake_bott));
PO4_lake= np.row_stack((PO4_lake_mean,PO4_lake_surf,PO4_lake_bott));
NO3_lake= np.row_stack((NO3_lake_mean,NO3_lake_surf,NO3_lake_bott)) ;   
Phy_lake= np.row_stack((Phy_lake_mean,Phy_lake_surf,Phy_lake_bott));
Zoo_lake= np.row_stack((Zoo_lake_mean,Zoo_lake_surf,Zoo_lake_bott));
Fish_lake= np.row_stack((Fish_lake_mean,Fish_lake_surf,Fish_lake_bott));
Veg_lake= np.row_stack((Veg_lake_mean,Veg_lake_surf,Veg_lake_bott));
PO4S_lake= np.row_stack((PO4S_lake_mean,PO4S_lake_surf,PO4S_lake_bott));
NO3S_lake= np.row_stack((NO3S_lake_mean,NO3S_lake_surf,NO3S_lake_bott));
Ben_lake= np.row_stack((Ben_lake_mean,Ben_lake_surf,Ben_lake_bott));

# assemble all data together
Ave_lake = np.row_stack((Tm_lake,O2_lake,PO4_lake,NO3_lake,Phy_lake,Zoo_lake,Fish_lake,
                        Veg_lake,PO4S_lake,NO3S_lake,Ben_lake))
df_lake  = DataFrame(Ave_lake,columns=['2m','5m','10m','20m'],index=['Tm','Tm_surf','Tm_bott',
                                                                 'O2','O2_surf','O2_bott',
                                                                 'PO4W','PO4W_surf','PO4W_bott',
                                                                 'NO3W','NO3W_surf','NO3W_bott',
                                                                 'Phy','Phy_surf','Phy_bott',
                                                                 'Zoo','Zoo_surf','Zoo_bott',
                                                                 'Fish','Fish_surf','Fish_bott',
                                                                 'Veg','Veg_surf','Veg_bott',
                                                                 'PO4S','PO4S_surf','PO4S_bott',
                                                                 'NO3S','NO3S_surf','NO3S_bott',
                                                                 'Ben','Ben_surf','Ben_bott'])

In [7]:
# Re-structure data
sum_2m =  pd.concat([df_0d.loc[:,'2m'],df_1d.loc[:,'2m'],df_lake.loc[:,'2m']],axis=1,keys=['0d','1d','lake'])
sum_5m =  pd.concat([df_0d.loc[:,'5m'],df_1d.loc[:,'5m'],df_lake.loc[:,'5m']],axis=1,keys=['0d','1d','lake'])
sum_10m =  pd.concat([df_0d.loc[:,'10m'],df_1d.loc[:,'10m'],df_lake.loc[:,'10m']],axis=1,keys=['0d','1d','lake'])
sum_20m =  pd.concat([df_0d.loc[:,'20m'],df_1d.loc[:,'20m'],df_lake.loc[:,'20m']],axis=1,keys=['0d','1d','lake'])
#pd.concat([df_0d.loc[:,'2m'],df_1d.loc[:,'2m'],df_lake.loc[:,'2m']],axis=1,keys=['0d','1d','lake'])

In [8]:
Total_annual=pd.concat([sum_2m,sum_5m,sum_10m,sum_20m],axis=1,keys=['2m','5m','10m','20m'])
Total_annual.to_csv('summer_averag.csv')